# Pull a small set of data and overfit on it

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image, ImageFilter
import random
import cv2
import os, glob

t = pd.read_csv('../data/all_data_info.csv'); t.head()

# filter down to just the first set of training data
train_1 = t[t['new_filename'].str.startswith('1')]
train_1 = train_1[train_1['in_train']]

# print(train_1.shape)

# pull a random sample of 200 impressionsim and 200 realism paintings
df1 = train_1[train_1['style'].str.startswith('Impre', na=False)].sample(n=200)
df2 = train_1[train_1['style'].str.startswith('Reali', na=False)].sample(n=200)

# split data into train and val
train_df1 = df1.sample(n=180)
train_df2 = df2.sample(n=180)

val_df1 = df1.loc[~df1.index.isin(train_df1.index)]
val_df2 = df2.loc[~df2.index.isin(train_df2.index)]

# print(val_df1.shape)
# print(train_df1.shape)

train_df = pd.concat([train_df1, train_df2])
val_df = pd.concat([val_df1, val_df2])

# print(train_df.shape)
# print(val_df.shape)

(360, 12)
(40, 12)


In [12]:
# load in imags with cv2, then convert to matrices that work with pytorch

for i in train_df1['new_filename']:
    

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.models as models
import torchvision.transforms as T

import numpy as np
import timeit

# Return dataloaders for train, test, and val
def load_images(train_df, test_df, val_df):
    # load the image files and store as 3-channel tensors
    
    # create the dataloaders
    train = data_utils.TensorDataset(X, y)
    loader_train = DataLoader(train, batch_size=64, shuffle=True)

    test = data_utils.TensorDataset(X, y)
    loader_test = DataLoader(test, batch_size=64, shuffle=True)
    
    val = data_utils.TensorDataset(X, y)
    loader_val = DataLoader(val, batch_size=64, shuffle=True)    
    
    return [loader_train, load_test, loader_val]


class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # read in N, C, H, W
        return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

dtype = torch.FloatTensor
print_every = 100

def reset(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

def train(model, loss_fn, optimizer, loader_train, num_epochs = 1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        for t, (x, y) in enumerate(loader_train):
            x_var = Variable(x.type(gpu_dtype))
            y_var = Variable(y.type(gpu_dtype).long())
            scores = model(x_var)
            loss = loss_fn(scores, y_var)
            if (t + 1) % print_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data[0]))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
def check_accuracy(model, loader):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for x, y in loader:
        x_var = Variable(x.type(gpu_dtype), volatile=True)
        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [ ]:
# Load data here using load_images

In [ ]:
# Create a simple network on our simplified dataset - should overfit it

simple_model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, stride=1),
    nn.Conv2d(3, 32, kernel_size=3, stride=2), # -> 110
    nn.ReLU(inplace=True),
    nn.BatchNorm2d(32),
    nn.MaxPool2d(2), # -> 55
    nn.Conv2d(32, 32, kernel_size=3, stride=2), # -> 27
    nn.ReLU(inplace=True),
    nn.BatchNorm2d(32),
    nn.MaxPool2d(2), # -> 13
    Flatten(),
    nn.Linear(5408, 100),
    nn.ReLU(inplace=True),
    nn.Linear(100, 2),
)

simple_model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimize = optim.Adam(simple_model.parameters(), lr=1e-2)

In [ ]:
# Now we're going to feed a random batch into the model and make sure the output is the right size
x = torch.randn(64, 3, 224, 224).type(dtype)
x_var = Variable(x.type(dtype)) # Construct a PyTorch Variable out of your input data
ans = fixed_model(x_var)        # Feed it through the model! 

# Check to make sure what comes out of your model
# is the right dimensionality... this should be True
# if you've done everything correctly
np.array_equal(np.array(ans.size()), np.array([64, 2]))

In [ ]:
# Overfit a resnet via transfer learning on this small set of data